### Load source code

In [105]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [106]:
import sys
sys.path.append('../')

In [149]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator
import source.belief as belief

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import shutil
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [150]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [151]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [185]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)
reload(belief)

<module 'source.belief' from '..\\source\\belief.py'>

### Set batch path, batch folder need not to already exist

In [186]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Multi_prof" #insert folder where batch is
batch_name = "test1" #insert batch name
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Parsing the csv to generate a batch containing the configurations and running them

In [187]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [188]:
if os.path.isdir(experiments_folder + exp_subfolder + "/" + batch_name):
    shutil.rmtree(experiments_folder + exp_subfolder + "/" + batch_name)
b.parse_batch()

In [189]:
batches = []
batches.append(b)

In [190]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

### Collecting results of the runs

In [191]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"],
                      "tb": c.stats["theoretical_bound"]}
                     for c in confs]
            util.plot_dicts(dlist, ylabel="$exp\_reg(t)$", xlabel="$t$", save=True, show=False, semilog=False,
                            path=b.results_folder_path,
                            title=confs[0].game.players[0].__class__.name + " vs " + confs[0].game.players[1].__class__.name)

ZeroDivisionError: division by zero

### Checking everything was fine by running the configuration "step-by-step"

In [192]:
gm = deepcopy(batches[0].configurations[0].game)

In [193]:
e = runner.Experiment(gm)

In [194]:
for i in range(50):
    e.run_multi_profile_interaction()
    #e.run_partial_feedback_interaction()
    #e.run_interaction()
    print(e.game.players[1])
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [163]:
print(gm.players[0].belief.loglk)

{(<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.2729841680429828, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.3984172590529413, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.3250210195694612, (<StochasticAttacker id:1 resources:1>, <StackelbergAttacker id:1 resources:1>): -0.7599894782683073, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.2811185803976612, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -0.9852982359988631, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.2544455091754825, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.3205924467048396, (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): -1.342946264648846, (<StochasticAttacker id:1 resources:1>, <StackelbergAttacker id:1 resources:1>): -0.941184

In [164]:
print(gm.profile_distribution)

(((<StochasticAttacker id:1 resources:1>, 0.7), (<StackelbergAttacker id:1 resources:1>, 0.3)),)


In [165]:
print(gm.profiles_history)

[['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sta1'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002'], ['sta1'], ['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.2

In [166]:
print(gm.players[0].learning_history)

['sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sta1', 'sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sto1-0.521-0.007-0.018-0.19-0.264', 'sta1|sto1-0.521-0.007-0.018-0.19-0.264', 'sta1|sto1-0.521-0.007-0.018-0.19-0.264', 'sta1|sto1-0.521-0.007-0.018-0.19-0.264', 'sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sta1', 'sto1-0.1861861861861862-0.19319319319319322-0.29129129129129133-0.07707707707707709-0.2522522522522523|sta1', 'sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sta1', 'sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sta1', 'sto1-0.029000000000000005-0.15900000000000003-0.48600000000000004-0.11100000000000002-0.21500000000000002|sta1', 'sto1-0.029000000000000005-0.15900000000000003-0.

In [167]:
print(gm.players[0].belief.freq)

{(<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [27, 23], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [24, 26], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [21, 29], (<StochasticAttacker id:1 resources:1>, <StackelbergAttacker id:1 resources:1>): [32, 18], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [24, 26], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [25, 25], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [18, 32], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [41, 9], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [44, 6], (<StochasticAttacker id:1 resources:1>, <StackelbergAttacker id:1 resources:1>): [32, 18], (<StochasticAttacker id:1 resources:1>, <StochasticAttacker id:1 resources:1>): [41, 9], (<Stochast

In [148]:
print(gm.profile_distribution)

(((<StochasticAttacker id:1 resources:1>, 0.7), (<StackelbergAttacker id:1 resources:1>, 0.3)),)


In [195]:
print(e.environment.last_exp_loss)

0.6597999999999999


In [196]:
print(e.environment.last_opt_loss)

None
